# Roadmap

The first generally released version of `meillionen` aims to provide

### `meillionen-mt`

- support f32, f64, string and time stamp datatypes in NetCDF
- add memory backend (will be a Python class with a function to convert to a NetCDF store)
- unit of measure and storage type mismatches make no attempt to coerce values, just fail with a useful error message
- cli interface for running models

    ```python
    import meillionen_cli as cli
    from landlab import Overland

    if __name__ == '__main__':
     import sys
     ov = Overland()
     ov.initialize_from_cli(sys.argv)
     ov.update()
     ov.finalize()
    ```

Passing information to a model will be done by config files. A config file could look something like

```yaml
---
model:
  name: 'simplecrop'
  variables:
    - source: 'surface_water__depth'
      target: 'infiltration_water__depth'
      store: 'overland'
  stores:
    - name: 'overland'
      file: 'overland.nc'
```

The example of Overland and SimpleCrop coupling will have:

- SimpleCrop, Overland will be in their own Docker images and will be accessible as command line programs
- SimpleCrop will support all its configuration options
- SimpleCrop's output data will be compatible with `meillionen`'s store API
- Show how replacement of OverlandFlow with `r.watershed` in GRASS is seamless
- Analysis of workflow results will be shown so that users can see how to make use of `meillionen` stores outside
  of an already wrapped model

Soon after subsequent versions aim to provide `meillionen-orchestrator`. Subsequent versions of `meillionen-mt` will
work on moving specific store implementations to their own packages like `meillionen-store-netcdf` and
`meillionen-store-arrow`.

### `meillionen-orchestrator`

`meillionen-orchestrator` will be library to wire components together and run the model by delegating to some scheduler
library (since model components have many initialization configuration options it will help to have one place to change
a coupled model run).

```python
import meillionen_orchestrator as mo
import pandas as pd

def get_daily_data(path):
    pass

# csdms/landlab is a fat image containing pymt/meillionen compatible versions of landlab models
ov = mo.ModelRef(image='csdms/landlab', model='Overland')
# agmip/dssat is a fat image containing pymt/meillionen compatible versions of dssat models
sc = mo.ModelRef(image='agmip/dssat', model='SimpleCrop')

coupled = mo.DagCoupledModel()
coupled.add_model('ov', ov)
coupled.add_model('sc', sc)
coupled.add_edge('ov', 'surface_water__depth', 'sc', 'infiltration_water_depth')
coupled.set_scheduler(mo.LuigiScheduler())
# options that enable configuration setup
coupled.add_option('daily_data', get_daily_data, options={'target': 'sc'})

if __name__ == '__main__':
  import sys
  coupled.initialize_from_cli(sys.argv, exit_on_fail=True)
  coupled.update()
  coupled.finalize()
```